### Imports

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set()

### Load data

In [2]:
train = pd.read_csv("../../data/home_default/train.csv")
test  = pd.read_csv("../../data/home_default/test.csv")

print("Shape of train:", train.shape)
print("Shape of test:",  test.shape)

Shape of train: (307511, 122)
Shape of test: (48744, 121)


### Merge train and test data

In [3]:
full = pd.concat([train, test])
train_N = len(train)

### Fill categorical NaN

In [4]:
for col in ('NAME_CONTRACT_TYPE', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY',
       'NAME_TYPE_SUITE', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE',
       'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'OCCUPATION_TYPE',
       'WEEKDAY_APPR_PROCESS_START', 'ORGANIZATION_TYPE', 'FONDKAPREMONT_MODE',
       'HOUSETYPE_MODE', 'WALLSMATERIAL_MODE', 'EMERGENCYSTATE_MODE'):
    full[col] = full[col].fillna(full[col].mode()[0])

### Fill numeric NaN

In [5]:
missing_cols = [col for col in full.columns if any(full[col].isnull())]
for col in missing_cols:
    full[col] = full[col].fillna(full[col].median())

### Confirm missing values are gone

In [6]:
full.isnull().sum().sum()

0

### Get dummies

In [7]:
full = pd.get_dummies(full)

### Separate data back into train and test

In [8]:
train = full[:train_N]
test = full[train_N:]
full.head()

,AMT_ANNUITY,AMT_CREDIT,AMT_GOODS_PRICE,AMT_INCOME_TOTAL,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_YEAR,...,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,WEEKDAY_APPR_PROCESS_START_FRIDAY,WEEKDAY_APPR_PROCESS_START_MONDAY,WEEKDAY_APPR_PROCESS_START_SATURDAY,WEEKDAY_APPR_PROCESS_START_SUNDAY,WEEKDAY_APPR_PROCESS_START_THURSDAY,WEEKDAY_APPR_PROCESS_START_TUESDAY,WEEKDAY_APPR_PROCESS_START_WEDNESDAY
0,24700.5,406597.5,351000.0,202500.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0,1,0,0,0,0,0,0,0,1
1,35698.5,1293502.5,1129500.0,270000.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,1,0,0,0,0,0
2,6750.0,135000.0,135000.0,67500.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,0,0,0,1,0,0,0,0,0
3,29686.5,312682.5,297000.0,135000.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1,0,0,0,0,0,0,0,0,1
4,21865.5,513000.0,513000.0,121500.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,0,0,0,0,0,0,1,0,0


### Split again into predictors, target, and id

In [9]:
train_y = train.TARGET
train_x = train.drop(["TARGET", "SK_ID_CURR"], axis=1)

test_id = test.SK_ID_CURR
test_x  = test.drop(["TARGET", "SK_ID_CURR"], axis=1)

### Imports and scoring helper function

In [10]:
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor

from sklearn.model_selection import cross_val_score, StratifiedKFold

kfold = StratifiedKFold()

def score_model(model):
    score = cross_val_score(model, train_x, train_y, cv=kfold, scoring="accuracy")
    print(score)
    print("\nAverage is ...")
    print(sum(score) / len(score))

### LGBM Regressor

In [11]:
lgbm_model = LGBMRegressor()
lgbm_model.fit(train_x, train_y)
# score_model(lgbm_model)

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
       learning_rate=0.1, max_depth=-1, min_child_samples=20,
       min_child_weight=0.001, min_split_gain=0.0, n_estimators=100,
       n_jobs=-1, num_leaves=31, objective=None, random_state=None,
       reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
       subsample_for_bin=200000, subsample_freq=1)

### XGB Regressor

In [1]:
# xgb_model = XGBRegressor()
# xgb_model.fit(train_x, train_y)
# score_model(xgb_model)

### Get predictions

In [14]:
predictions = lgbm_model.predict(test_x)

### Restrict predictions to appropriate range

In [21]:
predictions = np.clip(predictions, 0, 1)

### Sanity check

In [23]:
any(predictions < 0) or any(predictions > 1)

False

### Save file to CSV

In [25]:
pd.DataFrame({
    "SK_ID_CURR": test_id,
    "TARGET": predictions
}).to_csv("../../submissions/lgbm_regressor.csv", index=False)